In [ ]:
import ray
ray.init()
# ray.shutdown()

# Imports

In [5]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 40)
# pd.set_option('display.width', 1000)


import pandas_bokeh
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

import polygon_ds as pds
import polygon_df as pdf
import polygon_s3 as ps3
import ray_symbol_details as rsd
import market_preprocess as mp
import market_cov as mcov
import market_cluster as mc
import utils_pickle as up
# import ipdb
# ipdb.set_trace(context=10)

# Cluster market symbols

In [2]:
start_date = '2018-10-24'
end_date = '2020-11-24'

# load and pre-process market daily data

r = mp.prepare_data(start_date, end_date, beta_symbol='VTI')

In [5]:
X = r['g_zs_log_returns_resid']
Xt = X.transpose()
gt_industry = r['sym_meta'].industry.values
gt_sector = r['sym_meta'].sector.values

# compute covariance/correlation/distance/similarity matrics

dist_mat, cor_mat = mcov.cov_denoise_detone_dist(X, detone=False)
abs_cor_mat = abs(cor_mat)

In [5]:
# fit cluster models and evalue vs. ground truth labels

kclust = mc.cluster_ground_truth_eval(mc.get_kmean_clusters, 200, X, gt_industry)
print(kclust['adj. multual information'])

hclust = mc.cluster_ground_truth_eval(mc.get_hira_clusters, 200, Xt, gt_industry)
print(hclust['adj. multual information'])

In [220]:
# compute cluster descriptive stats

out = mc.cluster_metrics(lables=hclust['cluster_labels'], dist_mat=dist_mat, sym_meta=r['sym_meta'])

In [275]:
size_mask = (out.cluster_size > 5)

# size_mask = out.cluster_size > out.cluster_size.quantile(q=0.2)

similartiy_mask = out.avg_similartiy > out.avg_similartiy.quantile(q=0.25)

sector_purity_mask = out.sector_purity > out.sector_purity.quantile(q=0.25)

industry_purity_mask = out.industry_purity > out.industry_purity.quantile(q=0.25)

range_value_mask = out.avg_range_value_pct > out.avg_range_value_pct.quantile(q=0.75)


In [282]:
out[range_value_mask & similartiy_mask & size_mask].sort_values('avg_similartiy', ascending=False)[:9]

,names,symbols,cluster_size,avg_similartiy,avg_similartiy_std,sector_purity,industry_purity,top_sector,top_industry,avg_range_value_pct,med_daily_dollar_volume,dist_mat,symbol_meta
label,,,,,,,,,,,,,
137,"[First Majestic Silver Corp. (Canada), Alexco ...","[AG, AXU, BVN, CDE, EXK, FSM, GORO, GSS, HL, M...",14,0.465044,0.981774,0.500000,0.500000,Basic Materials,Metals & Mining,0.049257,5136780.000000,symbol AG AXU BVN CDE ...,na...
73,[Banco Macro S.A. ADR (representing Ten Class ...,"[BMA, CEPU, CRESY, EDN, GGAL, IRS, LOMA, PAM, ...",12,0.443925,0.979992,0.142857,0.142857,Utilities,Utilities - Regulated,0.052335,2342614.000000,symbol BMA CEPU CRESY EDN ...,...
22,"[Apache Corporation, Bonanza Creek Energy Inc....","[APA, BCEI, CDEV, CLR, CPE, CXO, DVN, EOG, EST...",33,0.433105,0.973471,0.500000,0.200000,Energy,Oil & Gas - E&P,0.048383,37959496.000000,symbol APA BCEI CDEV CLR ...,name ...
139,"[Antero Resources Corporation, CNX Resources C...","[AR, CNX, COG, CRK, EQT, RRC, SWN]",7,0.425349,0.974612,1.000000,1.000000,Energy,Oil & Gas - E&P,0.055165,50263184.000000,symbol AR CNX COG CRK ...,name sector ...
102,"[Archrock Inc., DMC Global Inc., CIRCOR Intern...","[AROC, BOOM, CIR, CLB, FTI, HAL, HLX, HP, KOS,...",29,0.416691,0.979757,0.333333,0.166667,Energy,Oil & Gas - Services,0.046271,12410298.000000,symbol AROC BOOM CIR CLB ...,...
85,"[Canadian Solar Inc., DAQO New Energy Corp. Am...","[CSIQ, DQ, ENPH, FSLR, JKS, RUN, SEDG, SPWR, T...",9,0.404275,0.978724,0.500000,0.500000,Technology,Semiconductors,0.047876,23333788.000000,symbol CSIQ DQ ENPH FSLR ...,...
53,"[Altimmune Inc., Alpha Pro Tech Ltd., Inovio P...","[ALT, APT, INO, LAKE, NNVC, NVAX]",6,0.362604,0.974705,0.500000,0.250000,Healthcare,Biotechnology,0.062078,720513.625000,symbol ALT APT INO LAKE ...,name ...
70,[Cheetah Mobile Inc. American Depositary Share...,"[CMCM, LX, NOAH, QD, SOGO, SOHU, XNET, YRD]",8,0.360847,0.982070,0.500000,0.250000,Technology,Credit Services,0.047916,4707837.000000,symbol CMCM LX NOAH QD ...,...
78,"[Arbutus Biopharma Corporation, ACADIA Pharmac...","[ABUS, ACAD, APLS, ARWR, ATNX, ATRA, BCRX, BPM...",52,0.359198,0.985155,1.000000,0.333333,Healthcare,Biotechnology,0.049116,10673644.000000,symbol ABUS ACAD APLS ARWR ...,name se...


In [287]:
out.iloc[137, :].symbol_meta.index

Index(['AG', 'AXU', 'BVN', 'CDE', 'EXK', 'FSM', 'GORO', 'GSS', 'HL', 'MAG',
       'MUX', 'PVG', 'SVM', 'USAS'],
      dtype='object', name='symbol')

# Corex

In [ ]:
corex300 = up.pickle_load('data/corex300.pkl')

# X = df.reset_index(drop=True)
# corex, full_df = mc.corex_fit(X, n_hidden=200)

# up.pickle_dump((corex, full_df), file_name='corex200.pkl')

# gsutil copy /Users/bobcolner/QuantClarity/pandas-polygon/corex200.pkl gs://emerald-skill-datascience/

# GLRM

In [ ]:
import h2o
from h2o.estimators import H2OGeneralizedLowRankEstimator
h2o.init()

# Import the USArrests dataset into H2O:
arrestsH2O = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/pca_test/USArrests.csv")

# Split the dataset into a train and valid set:
train, valid = arrestsH2O.split_frame(ratios=[.8], seed=1234)

# Build and train the model:
glrm_model = H2OGeneralizedLowRankEstimator(
    k=4,
    loss="Huber",
    loss_by_col={'cat_var': 'Categorical'},
    regularization_x="OneSparse", # None (default), Quadratic, L2, L1, NonNegative, OneSparse, UnitOneSparse, Simplex.
    regularization_y="OneSparse",
    gamma_x=0.5,
    gamma_y=0.5,
    max_iterations=1000,
    recover_svd=True,
    init="PlusPlus",
    transform="standardize",
   )
glrm_model.train(training_frame=train)